In [12]:
import os
import pandas as pd
import re

def clean_company_name(company):
    """
    clean the company name by converting to lowercase,
    removing punctuation, and extra spaces.
    """
    s = str(company).strip().lower()
    s = re.sub(r'[^a-z0-9\s]', '', s)
    s = re.sub(r'\s+', ' ', s)
    return s

# Load the processed stock dataset
processed_path = "../data/processed/korean_stock_data_cleaned.csv"
df = pd.read_csv(processed_path, parse_dates=["date"])
print(f"Loaded {len(df)} rows from {processed_path}.")

# Load the ticker mapping CSV
ticker_path = "../data/raw/ticker.csv"
df_ticker = pd.read_csv(ticker_path)
print(f"Loaded {len(df_ticker)} rows from {ticker_path}.")

# Clean company names in both datasets
df["company_clean"] = df["company"].apply(clean_company_name)
df_ticker["company_clean"] = df_ticker["company"].apply(clean_company_name)

# Create a mapping dictionary from cleaned company name to ticker
ticker_mapping = dict(zip(df_ticker["company_clean"], df_ticker["ticker"]))

# Map each row's cleaned company name to a ticker; if no match, return None.
df["ticker"] = df["company_clean"].apply(lambda x: ticker_mapping.get(x, None))

# Count how many rows got a match and how many did not
total_rows = len(df)
matched_count = df["ticker"].notnull().sum()
not_matched_count = total_rows - matched_count

print(f"Total rows: {total_rows}")
print(f"Matched tickers: {matched_count}")
print(f"Not matched tickers: {not_matched_count}")

# Optionally, drop rows with unknown tickers
df_matched = df[df["ticker"].notnull()].copy()

# Save the updated dataset with tickers to a new file
output_path = "../data/mapping/cleaned_stock_dataset.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_matched.to_csv(output_path, index=False)
print(f"Updated dataset with tickers saved to {output_path}")


Loaded 868429 rows from ../data/processed/korean_stock_data_cleaned.csv.
Loaded 2587 rows from ../data/raw/ticker.csv.
Total rows: 868429
Matched tickers: 249251
Not matched tickers: 619178
Updated dataset with tickers saved to ../data/mapping/cleaned_stock_dataset.csv


In [13]:
import os
import pandas as pd
import re

def clean_company_name(company):
    """
    clean the company name by converting to lowercase,
    removing punctuation, and extra spaces.
    """
    s = str(company).strip().lower()
    s = re.sub(r'[^a-z0-9\s]', '', s)
    s = re.sub(r'\s+', ' ', s)
    return s

# Load the processed cleaned dataset that still has the company column
df_clean = pd.read_csv("../data/processed/korean_stock_data_cleaned.csv", parse_dates=["date"])
print(f"Loaded {len(df_clean)} rows from the cleaned dataset.")

# Clean the company names in the cleaned dataset
df_clean["company_clean"] = df_clean["company"].apply(clean_company_name)

# Load the ticker mapping from the CSV file
df_ticker = pd.read_csv("../data/raw/ticker.csv")
print(f"Loaded {len(df_ticker)} rows from ticker mapping.")

# Clean company names in the ticker mapping
df_ticker["company_clean"] = df_ticker["company"].apply(clean_company_name)

# Build a mapping dictionary from cleaned company name to ticker
ticker_mapping = dict(zip(df_ticker["company_clean"], df_ticker["ticker"]))

# Map each row in the cleaned dataset to a ticker (or None if no match)
df_clean["ticker"] = df_clean["company_clean"].apply(lambda x: ticker_mapping.get(x, None))

# Filter rows where ticker is missing
df_unmatched = df_clean[df_clean["ticker"].isnull()]

# Group by the cleaned company name and count the unmatched occurrences
unmatched_counts = df_unmatched.groupby("company_clean").size().reset_index(name="count")

print("Companies without ticker mapping:")
print(unmatched_counts.sort_values(by="count", ascending=False))

# Optionally, you can also see the total number of unmatched companies:
total_unmatched = unmatched_counts.shape[0]
print(f"\nTotal unmatched companies: {total_unmatched}")


Loaded 868429 rows from the cleaned dataset.
Loaded 2587 rows from ticker mapping.
Companies without ticker mapping:
              company_clean  count
0                   ace bed   3089
122        samwha capacitor   3089
104  namyang dairy products   3089
105              nexen tire   3089
106            next science   3089
..                      ...    ...
164                unkown18    324
209                unkown59    324
162                unkown16    291
205                unkown55    153
224                unkown72     67

[249 rows x 2 columns]

Total unmatched companies: 249
